In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow as tf

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

2023-10-03 22:42:57.478145: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/lib/python3.11/site-packages/h5py/__init__.py:36: UserWarning: h5py is running against HDF5 1.14.2 when it was built against 1.14.1, this may cause problems
  _warn(("h5py is running against HDF5 {0} when it was built against {1}, "


In [2]:
train = pd.read_csv('./jigsaw-toxic-comment-train.csv')
valid = pd.read_csv('./validation.csv')
test = pd.read_csv('./test.csv')

In [3]:
train.drop(['severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'],axis=1, inplace=True)

In [4]:
train.shape

(223549, 3)

In [5]:
train['comment_text'].apply(lambda x: len(str(x).split())).max()

2321

In [6]:
from sklearn.metrics import roc_curve, auc

In [15]:
def roc_auc(predictions, target):
    fpr, tpr, thresholds = roc_curve(target, predictions)
    roc_auc = auc(fpr, tpr)
    return roc_auc

In [8]:
x_train, x_valid, y_train, y_valid = train_test_split(
    train.comment_text.values, train.toxic.values,
    stratify = train.toxic.values,
    random_state = 42,
    test_size = 0.2, shuffle = True
)

In [9]:
from keras.preprocessing import text
from keras.utils import pad_sequences

token = text.Tokenizer(num_words=None)
max_len = 2500

token.fit_on_texts(list(x_train) + list(x_valid))
x_train_seq = token.texts_to_sequences(x_train)
x_valid_seq = token.texts_to_sequences(x_valid)

x_train_pad = pad_sequences(x_train_seq, maxlen=max_len)
x_valid_pad = pad_sequences(x_valid_seq, maxlen=max_len)

word_index = token.word_index

In [10]:
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense, Embedding

In [11]:
%%time
model = Sequential()
model.add(Embedding(len(word_index)+1, 300, input_length=max_len))
model.add(SimpleRNN(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

2023-10-03 22:43:12.188620: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2023-10-03 22:43:12.188645: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: gallirium
2023-10-03 22:43:12.188650: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: gallirium
2023-10-03 22:43:12.188802: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 535.113.1
2023-10-03 22:43:12.188820: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 535.113.1
2023-10-03 22:43:12.188824: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:309] kernel version seems to match DSO: 535.113.1
2023-10-03 22:43:12.191308: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune u

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 2500, 300)         90077400  
                                                                 
 simple_rnn (SimpleRNN)      (None, 100)               40100     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 90,117,601
Trainable params: 90,117,601
Non-trainable params: 0
_________________________________________________________________
CPU times: user 1.21 s, sys: 1.25 s, total: 2.46 s
Wall time: 316 ms


In [12]:
model.fit(x_train_pad, y_train, epochs=1, batch_size=256)

699/699 [==============================] - 2011s 3s/step - loss: 0.2202 - accuracy: 0.9266


In [13]:
scores = model.predict(x_valid_pad)

1398/1398 [==============================] - 390s 279ms/step


In [16]:
print(f"AUC: {roc_auc(scores, y_valid)}")

AUC: 0.8962901335849808


In [17]:
## We can do better!!
## Maybe LSTMs or BERT..?